In [25]:

print("----- Missing Value Analysis (Pre-Cleaning) -----")

missing_values = df.isnull().sum()

missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_percentage = missing_percentage.sort_values(ascending=False)

print("\n----- Top 15 Missing Percentage (Sorted) -----")
print(missing_percentage.head(15))

----- Missing Value Analysis (Pre-Cleaning) -----

----- Top 15 Missing Percentage (Sorted) -----
Start_Time               9.616047
End_Time                 9.616047
Month                    9.616047
Accident_Duration_Min    9.616047
Hour                     9.616047
Weekday                  9.616047
Source                   0.000000
ID                       0.000000
Description              0.000000
Street                   0.000000
City                     0.000000
County                   0.000000
Severity                 0.000000
Start_Lat                0.000000
Start_Lng                0.000000
dtype: float64


In [29]:

print("----- DROPPING COLUMNS (>40% Missing) -----\n")

missing_percentage = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)

drop_threshold = 40
columns_to_drop = missing_percentage[missing_percentage > drop_threshold].index

df.drop(columns=columns_to_drop, inplace=True)

print(f"✔ Dropped {len(columns_to_drop)} columns (e.g., End_Lat, End_Lng, etc.).")
print(f"New shape: {df.shape}\n")

----- DROPPING COLUMNS (>40% Missing) -----

✔ Dropped 0 columns (e.g., End_Lat, End_Lng, etc.).
New shape: (7728394, 48)



In [28]:

print("-----  Handling Remaining Missing Values -----\n")

numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        
        df[col] = df[col].fillna(df[col].median())

print("✔ Missing values in numeric columns handled using MEDIAN.\n")

for col in categorical_cols:
    if df[col].isnull().sum() > 0:
       
        df[col] = df[col].fillna(df[col].mode()[0])

print("✔ Missing values in categorical columns handled using MODE.\n")

print("----- Remaining Missing Values After Filling -----")
print(df.isnull().sum().sort_values(ascending=False).head(10))

-----  Handling Remaining Missing Values -----

✔ Missing values in numeric columns handled using MEDIAN.

✔ Missing values in categorical columns handled using MODE.

----- Remaining Missing Values After Filling -----
Start_Time      743166
End_Time        743166
Source               0
ID                   0
Severity             0
Start_Lat            0
Start_Lng            0
Distance(mi)         0
Description          0
Street               0
dtype: int64


In [27]:


print("----- Converting Datetime Columns -----\n")

df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')

df['End_Time'] = pd.to_datetime(df['End_Time'], errors='coerce')

if 'Weather_Timestamp' in df.columns:
    df['Weather_Timestamp'] = pd.to_datetime(df['Weather_Timestamp'], errors='coerce')

print("✔ Datetime conversion completed.\n")

print(df[['Start_Time', 'End_Time']].dtypes)

----- Converting Datetime Columns -----

✔ Datetime conversion completed.

Start_Time    datetime64[ns]
End_Time      datetime64[ns]
dtype: object


In [31]:

print("----- Creating New Features -----\n")

df['Hour'] = df['Start_Time'].dt.hour


df['Weekday'] = df['Start_Time'].dt.day_name()


df['Month'] = df['Start_Time'].dt.month_name()

print("✔ Created Hour, Weekday, and Month columns.\n")

df['Accident_Duration_Min'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds() / 60

print("✔ Created Accident_Duration_Min column.\n")

print(df[['Start_Time', 'End_Time', 'Hour', 'Weekday', 'Month', 'Accident_Duration_Min']].head())

----- Creating New Features -----

✔ Created Hour, Weekday, and Month columns.

✔ Created Accident_Duration_Min column.

           Start_Time            End_Time  Hour Weekday     Month  \
0 2016-02-08 05:46:00 2016-02-08 11:00:00   5.0  Monday  February   
1 2016-02-08 06:07:59 2016-02-08 06:37:59   6.0  Monday  February   
2 2016-02-08 06:49:27 2016-02-08 07:19:27   6.0  Monday  February   
3 2016-02-08 07:23:34 2016-02-08 07:53:34   7.0  Monday  February   
4 2016-02-08 07:39:07 2016-02-08 08:09:07   7.0  Monday  February   

   Accident_Duration_Min  
0                  314.0  
1                   30.0  
2                   30.0  
3                   30.0  
4                   30.0  


In [ ]:


print("----- SUPPLEMENTARY STEP: Handling Duplicates -----\n")


duplicate_rows = df.duplicated().sum()

if duplicate_rows > 0:
    print(f"Found {duplicate_rows} exact duplicate rows.")
    
    
    df.drop_duplicates(inplace=True)
    
    print(f"✔ Removed duplicates. New shape: {df.shape}\n")
else:
    print("No exact duplicate rows found. Skipping removal.\n")

----- SUPPLEMENTARY STEP: Handling Duplicates -----

No exact duplicate rows found. Skipping removal.



In [ ]:


print("----- SUPPLEMENTARY STEP: Handling Outliers (Duration) -----\n")

col = 'Accident_Duration_Min'


Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1


upper_bound = Q3 + 1.5 * IQR


outlier_count = df[df[col] > upper_bound].shape[0]

if outlier_count > 0:
    print(f"Found {outlier_count} extreme outliers in {col}.")
    
   
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
    
    print(f"✔ Capped extreme duration outliers at {upper_bound:.2f} minutes.\n")
else:
    print("No extreme outliers found for duration.\n")

----- SUPPLEMENTARY STEP: Handling Outliers (Duration) -----

Found 680043 extreme outliers in Accident_Duration_Min.
✔ Capped extreme duration outliers at 261.58 minutes.

